### Example creating the stiffness for a linear elasticity element using tensors or matrices

In [1]:
using JuAFEM
using ForwardDiff

In [2]:
E = 200e9
ν = 0.3
λ = E*ν / ((1+ν) * (1 - 2ν))
μ = E / (2(1+ν))
δ(i,j) = i == j ? 1.0 : 0.0
g(i,j,k,l) = λ*δ(i,j)*δ(k,l) + μ*(δ(i,k)*δ(j,l) + δ(i,l)*δ(j,k))

# Create a random symmetric material stiffness
C = SymmetricTensor{4, 2}(g)

Ee = [C[1,1,1,1] C[2,2,1,1] C[1,2,1,1];
      C[1,1,2,2] C[2,2,2,2] C[1,2,2,2];
      C[1,1,1,2] C[2,2,1,2] C[1,2,1,2]];

In [3]:
function_space = Lagrange{2, JuAFEM.Square, 1}()
quad_rule = get_gaussrule(Dim{2}, JuAFEM.Square(), 2)
fe_values = FEValues(Float64, quad_rule, function_space);

x = [0. 1 1 0;
     0 0 1 1]
x_vec = reinterpret(Vec{2, Float64}, x, (4,));

## Stiffness

In [4]:
function ke_element_mat!{T,dim}(Ke, X::Vector{Vec{2, T}}, fe_values::FEValues{dim}, Ee, B, DB, BDB)
    n_basefuncs = n_basefunctions(get_functionspace(fe_values))
    @assert length(X) == n_basefuncs
    
    reinit!(fe_values, X)

    for q_point in 1:length(JuAFEM.points(get_quadrule(fe_values)))
        
        for i in 1:n_basefuncs
            dNdx = shape_gradient(fe_values, q_point, i)[1]
            dNdy = shape_gradient(fe_values, q_point, i)[2]
            B[1, 2*i - 1] = dNdx
            B[2, 2*i - 0] = dNdy
            B[3, 2*i - 0] = dNdx
            B[3, 2*i - 1] = dNdy
        end
        
        A_mul_B!(DB, Ee, B)
        At_mul_B!(BDB, B, DB)
        scale!(BDB, detJdV(fe_values, q_point))
        for p in 1:size(Ke,1)
            for q in 1:size(Ke,2)
                Ke[p, q] += BDB[p, q]
            end
        end
    end
    
    return Ke
end

ke_element_mat! (generic function with 1 method)

In [5]:
B = zeros(3, 8)
DB = zeros(3,8)
BDB = zeros(8,8)
Ke = zeros(8,8)
fill!(Ke, 0.0)
ke_element_mat!(Ke, x_vec, fe_values, Ee, B, DB, BDB)

8x8 Array{Float64,2}:
  1.15385e11   4.80769e10  -7.69231e10  …   1.92308e10  -9.61538e9 
  4.80769e10   1.15385e11  -9.61538e9       9.61538e9   -7.69231e10
 -7.69231e10  -9.61538e9    1.15385e11     -5.76923e10   4.80769e10
  9.61538e9    1.92308e10  -4.80769e10      4.80769e10  -5.76923e10
 -5.76923e10  -4.80769e10   1.92308e10     -7.69231e10   9.61538e9 
 -4.80769e10  -5.76923e10   9.61538e9   …  -9.61538e9    1.92308e10
  1.92308e10   9.61538e9   -5.76923e10      1.15385e11  -4.80769e10
 -9.61538e9   -7.69231e10   4.80769e10     -4.80769e10   1.15385e11

In [6]:
function ke_elementt!{T,dim}(Ke, X::Vector{Vec{2, T}}, fe_values::FEValues{dim}, D)
    n_basefuncs = n_basefunctions(get_functionspace(fe_values))
    @assert length(X) == n_basefuncs
    reinit!(fe_values, X)
    for q_point in 1:length(JuAFEM.points(get_quadrule(fe_values)))
        for i in 1:n_basefuncs
            for j in 1:n_basefuncs
                ∇ϕi = shape_gradient(fe_values, q_point, i)
                ∇ϕj = shape_gradient(fe_values, q_point, j)
                Ke_e = (∇ϕj ⊗ ∇ϕi) ⊡ D * detJdV(fe_values, q_point)
                Ke[dim*(i-1) + 1:dim*i, dim*(j-1) + 1:dim*j] += Ke_e
            end
        end
    end
    return Ke
end

ke_elementt! (generic function with 1 method)

In [7]:
# Note the minor transpose!
C2 = Tensor{4, 2}((i,l,k,j) -> C[i,j,k,l])
x = [0. 1 1 0;
     0 0 1 1]
x_vec = reinterpret(Vec{2, Float64}, x, (4,))
Ke2 = zeros(8,8)
ke_elementt!(Ke2, x_vec, fe_values, C2)

8x8 Array{Float64,2}:
  1.15385e11   4.80769e10  -7.69231e10  …   1.92308e10  -9.61538e9 
  4.80769e10   1.15385e11  -9.61538e9       9.61538e9   -7.69231e10
 -7.69231e10  -9.61538e9    1.15385e11     -5.76923e10   4.80769e10
  9.61538e9    1.92308e10  -4.80769e10      4.80769e10  -5.76923e10
 -5.76923e10  -4.80769e10   1.92308e10     -7.69231e10   9.61538e9 
 -4.80769e10  -5.76923e10   9.61538e9   …  -9.61538e9    1.92308e10
  1.92308e10   9.61538e9   -5.76923e10      1.15385e11  -4.80769e10
 -9.61538e9   -7.69231e10   4.80769e10     -4.80769e10   1.15385e11

In [8]:
norm(Ke - Ke2) / norm(Ke)

8.632935774790041e-17